In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px

In [3]:
df = pd.read_csv('C:/Users/tijmen.vanetten/Documents/emobility-vanetten/data/processed/sessions_data_all.csv')
pois = pd.read_csv('C:/Users/tijmen.vanetten/Documents/emobility-vanetten/data/raw/GB_shell_recharge_locations.csv')
pois = pois[['LOCATION_ID', 'LATITUDE', 'LONGITUDE']]

C:\Users\tijmen.vanetten\AppData\Local\Temp\ipykernel_20244\504135515.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/tijmen.vanetten/Documents/emobility-vanetten/data/processed/sessions_data_all.csv')


In [4]:
df = df.merge(pois, on='LOCATION_ID', how='left')
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.LONGITUDE, df.LATITUDE))

In [5]:
gdf_grouped = gdf.groupby(by=['LOCATION_ID','DATE'])['ENERGY_DELIVERED_KWH'].sum().groupby('LOCATION_ID').mean()
gdf_grouped = gdf_grouped.to_frame().merge(pois, on='LOCATION_ID')

In [6]:
gdf_grouped['ENERGY_DELIVERED_KWH'].argmax()

3975

In [7]:
gdf_grouped.iloc[44]

LOCATION_ID             141264.000000
ENERGY_DELIVERED_KWH       178.215098
LATITUDE                    51.540729
LONGITUDE                   -0.102348
Name: 44, dtype: float64

In [8]:
import plotly.graph_objects as go

fig = px.scatter_mapbox(gdf_grouped, lat="LATITUDE", lon="LONGITUDE", hover_data=["ENERGY_DELIVERED_KWH"],
                        color_discrete_sequence=["fuchsia"], zoom=5, height=600,
                        center=go.layout.mapbox.Center(lat=54.23, lon=-4.5481),
                        color="ENERGY_DELIVERED_KWH", size='ENERGY_DELIVERED_KWH',
                        color_continuous_scale=px.colors.sequential.Plasma)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [9]:
gdf_grouped

,LOCATION_ID,ENERGY_DELIVERED_KWH,LATITUDE,LONGITUDE
0,141205,486.319011,51.550110,-0.108551
1,141206,231.049042,51.525217,-0.088930
2,141207,184.933738,51.516519,-0.291124
3,141211,274.379203,51.517888,-0.065300
4,141212,150.894379,52.472455,-1.912596
...,...,...,...,...
4020,1189225,4.890000,51.605665,-0.339054
4021,1189968,231.965250,52.126988,0.223338
4022,1191794,2.196000,51.631485,-0.175560
4023,1191795,38.568000,52.243858,0.709128
